In [19]:
!pip3 install numpy --upgrade
!pip3 install scipy --upgrade

In [20]:
import fft_project
from fft_project import base
import psutil
import logging
import time
import scipy
import numpy as np

In [21]:
'''Below are the testcases, where we controlled the multithreading and creating copies
of our version of 2DFFT versus Scipy fft2'''

#Printing out Current CPU Info
print("="*40, "CPU Info", "="*40)
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 2800.00Mhz
Min Frequency: 2800.00Mhz
Current Frequency: 2800.00Mhz


In [22]:
'''Function for creating testing matrices''' 
def random_matrix(n, m):
    logging.info(f'Matrix shape: {n}x{m}')
    result = np.zeros((n, m), dtype=np.complex128)
    result.real = np.random.rand(n, m)
    return result




In [23]:
'''No Multithreading, No Copies Created'''

class fft_tests:
    @base.testing.test
    def fft_cpp_impl_test():
        result = random_matrix(4096, 4096 * 2)
        start = time.time()
        expected = scipy.fft.fft2(result)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        start = time.time()
        result = fft_project.fft2d(result, return_copy=False, use_threads=False)
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        expected = abs(expected)
        result = abs(result)
        correctness = np.allclose(result, expected, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness

    @base.testing.test
    def inverse_test():
        a = random_matrix(4096 * 2, 4096)
        transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
        start = time.time()
        inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        inversed_our = transformed
        start = time.time()
        inversed_our = fft_project.fft2d(
            inversed_our, return_copy=False, use_threads=False, inverse=True
        )
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        inversed_scipy = abs(inversed_scipy)
        inversed_our = abs(inversed_our)
        correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness


if __name__ == '__main__':
    base.prepare_logger(level=logging.DEBUG)
    tests_list = [
        fft_tests.fft_cpp_impl_test,
        fft_tests.inverse_test,
    ]
    base.testing.run_tests(tests_list)

[INFO]  Executing 2 tests
[INFO]  TEST: fft_cpp_impl_test
[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 1.7005300521850586
[INFO]  Our: 1.0896670818328857
[INFO]  64.08% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  TEST: inverse_test
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.5195980072021484
[INFO]  Our: 1.0320971012115479
[INFO]  198.63% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  0 tests failed
[INFO]  SUCCEED!


In [24]:
'''Multithreading, No Copies'''

class fft_tests:
    @base.testing.test
    def fft_cpp_impl_test():
        result = random_matrix(4096, 4096 * 2)
        start = time.time()
        expected = scipy.fft.fft2(result)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        start = time.time()
        result = fft_project.fft2d(result, return_copy=False, use_threads=True)
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        expected = abs(expected)
        result = abs(result)
        correctness = np.allclose(result, expected, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness

    @base.testing.test
    def inverse_test():
        a = random_matrix(4096 * 2, 4096)
        transformed = fft_project.fft2d(a, return_copy=False, use_threads=True)
        start = time.time()
        inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        inversed_our = transformed
        start = time.time()
        inversed_our = fft_project.fft2d(
            inversed_our, return_copy=False, use_threads=True, inverse=True
        )
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        inversed_scipy = abs(inversed_scipy)
        inversed_our = abs(inversed_our)
        correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness


if __name__ == '__main__':
    base.prepare_logger(level=logging.DEBUG)
    tests_list = [
        fft_tests.fft_cpp_impl_test,
        fft_tests.inverse_test,
    ]
    base.testing.run_tests(tests_list)

[INFO]  Executing 2 tests
[INFO]  TEST: fft_cpp_impl_test
[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 1.7650959491729736
[INFO]  Our: 0.6212880611419678
[INFO]  35.20% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  TEST: inverse_test
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.5513339042663574
[INFO]  Our: 0.662175178527832
[INFO]  120.10% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  0 tests failed
[INFO]  SUCCEED!


In [25]:
'''No Multithreading, Creating Copies'''
class fft_tests:
    @base.testing.test
    def fft_cpp_impl_test():
        result = random_matrix(4096, 4096 * 2)
        start = time.time()
        expected = scipy.fft.fft2(result)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        start = time.time()
        result = fft_project.fft2d(result, return_copy=True, use_threads=False)
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        expected = abs(expected)
        result = abs(result)
        correctness = np.allclose(result, expected, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness

    @base.testing.test
    def inverse_test():
        a = random_matrix(4096 * 2, 4096)
        transformed = fft_project.fft2d(a, return_copy=True, use_threads=False)
        start = time.time()
        inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        inversed_our = transformed
        start = time.time()
        inversed_our = fft_project.fft2d(
            inversed_our, return_copy=True, use_threads=False, inverse=True
        )
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        inversed_scipy = abs(inversed_scipy)
        inversed_our = abs(inversed_our)
        correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness


if __name__ == '__main__':
    base.prepare_logger(level=logging.DEBUG)
    tests_list = [
        fft_tests.fft_cpp_impl_test,
        fft_tests.inverse_test,
    ]
    base.testing.run_tests(tests_list)


[INFO]  Executing 2 tests
[INFO]  TEST: fft_cpp_impl_test
[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 1.6747000217437744
[INFO]  Our: 1.5568809509277344
[INFO]  92.96% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  TEST: inverse_test
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.6708500385284424
[INFO]  Our: 1.4754376411437988
[INFO]  219.94% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  0 tests failed
[INFO]  SUCCEED!


In [26]:
'''Multithreaded, Creating copies'''
class fft_tests:
    @base.testing.test
    def fft_cpp_impl_test():
        result = random_matrix(4096, 4096 * 2)
        start = time.time()
        expected = scipy.fft.fft2(result)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        start = time.time()
        result = fft_project.fft2d(result, return_copy=True, use_threads=True)
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        expected = abs(expected)
        result = abs(result)
        correctness = np.allclose(result, expected, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness

    @base.testing.test
    def inverse_test():
        a = random_matrix(4096 * 2, 4096)
        transformed = fft_project.fft2d(a, return_copy=True, use_threads=True)
        start = time.time()
        inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
        scipy_time = time.time() - start
        logging.info(f'Scipy: {scipy_time}')
        inversed_our = transformed
        start = time.time()
        inversed_our = fft_project.fft2d(
            inversed_our, return_copy=True, use_threads=True, inverse=True
        )
        our_time = time.time() - start
        logging.info(f'Our: {our_time}')
        logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
        inversed_scipy = abs(inversed_scipy)
        inversed_our = abs(inversed_our)
        correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
        logging.info(f'Is the result correct: {correctness}')
        assert correctness


if __name__ == '__main__':
    base.prepare_logger(level=logging.DEBUG)
    tests_list = [
        fft_tests.fft_cpp_impl_test,
        fft_tests.inverse_test,
    ]
    base.testing.run_tests(tests_list)


[INFO]  Executing 2 tests
[INFO]  TEST: fft_cpp_impl_test
[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 1.6659619808197021
[INFO]  Our: 1.0258030891418457
[INFO]  61.57% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  TEST: inverse_test
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.5056707859039307
[INFO]  Our: 1.0125770568847656
[INFO]  200.24% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  SUCCEED!

[INFO]  0 tests failed
[INFO]  SUCCEED!
